<a href="https://colab.research.google.com/github/orangetin/OpenChatKit/blob/peft/training/lora/example/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenChatKit - Fine-tuning

### Check GPU availability

In [ ]:
!nvidia-smi

### Install conda

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh && chmod +x Miniconda3-latest-Linux-x86_64.sh && ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

### Setting up conda environment

In [ ]:
!conda install mamba -n base -c conda-forge -y

In [ ]:
!git clone https://github.com/orangetin/OpenChatKit.git --branch peft && cd OpenChatKit && mamba create -n OpenChatKit python=3.10.9 -y

In [ ]:
!source activate OpenChatKit && mamba install pytorch torchvision torchaudio cudatoolkit-dev pytorch-cuda=11.6 -c pytorch -c nvidia -c conda-forge -y

In [ ]:
!source activate OpenChatKit && export CUDA_HOME=$CONDA_PREFIX && pip install accelerate evaluate datasets peft chardet cchardet transformers git+https://github.com/EleutherAI/DeeperSpeed.git bitsandbytes && pip install 'transformers[sklearn]'

### Download dataset and convert jsonl to json

In [ ]:
!cd OpenChatKit/training/lora && mkdir data && mkdir data_jsonl

In [ ]:
!cd OpenChatKit/training/lora/data_jsonl && wget https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl

In [ ]:
import json

with open('OpenChatKit/training/lora/data_jsonl/unified_chip2.jsonl', 'r') as in_file:
    lines = [json.loads(line) for line in in_file.readlines()]

with open('OpenChatKit/training/lora/data/unified_chip2.json', 'w') as out_file:
    json.dump(lines, out_file)

### Initialize training in 8-bit

Edits config to disable fp16

In [ ]:
!cd OpenChatKit/training/lora && sed -i -e 's/"enabled": true,/"enabled": false,/g' example/config.json

To change to fp16, replace `--int8 \ --low_cpu_mem_usage \` with `--fp16 \`

In [ ]:
!source activate OpenChatKit && export CUDA_HOME=$CONDA_PREFIX && cd OpenChatKit/training/lora && deepspeed --num_gpus=1 finetune.py \
--deepspeed example/config.json \
--model_name_or_path togethercomputer/RedPajama-INCITE-Chat-3B-v1 \
--train_file data/unified_chip2.json \
--validation_split_percentage 10 \
--do_train \
--do_eval \
--overwrite_cache \
--evaluation_strategy="steps" \
--output_dir finetuned \
--num_train_epochs 1 \
--eval_steps 15 \
--gradient_accumulation_steps 2 \
--per_device_train_batch_size 4 \
--use_fast_tokenizer True \
--learning_rate 1e-5 \
--warmup_steps 10 \
--int8 \
--low_cpu_mem_usage \
--no_cache